# A study on the Iterative Prisoner's Dilemma

_Group 9 - Elia Bonetto, Filippo Rigotto, Luca Attanasio, Francesco Savio_

This notebook is a brief presentation of the code we developed for our project involving the Iterative Prisoner's Dilemma (IPD).

For more detailed informations, please see the [full report in PDF](report/report.pdf) and optionally the [supplementary material](report/report-suppl.pdf).

We start by defining the classes we built our code upon and then proceed to show many variations of the game.

## Base classes

### Player and Strategy

We defined a class Player with different methods that aim to store the history of moves and plays against other players.

We also defined a base class Strategy to represent all strategies.
From this base class, we derived the ProbStrategy class to represent through a parameter $k$ those strategies where cooperation occurs with a given probability (namely _Bad guy_, _MainlyBad_, _Nice guy_, _MainlyNice_, _Indifferent_).
Tit-for-Tat (_TfT_), Tit-for-2-Tat (_Tf2T_) and Grim-Trigger (_GrT_) strategies are also derived from the base Strategy class.

These are all the strategies we considered in our work, even if many more exist and could be used.

In [ ]:
%load code/strategy.py

In [ ]:
%load code/player.py

### Matrix Generator function

Even if in our tests we used the default payoff matrix, we defined a function to generate a valid matrix to be used for the game.

In [ ]:
%load code/mgen.py

In [ ]:
%run code/mgen.py
generatePayoffMatrix()

## Analyzed scenarios

### IPD between two players

For the simple IPD between two players, each player plays against all others and itself.

Since in this case every player has a different fixed strategy, it is the same as checking how each strategy performs against itself and all the others. For each match, we iterate the game `NUM_REPETITIONS` times in order to have some statistics on the final result, namely mean and standard deviation. 

We plot these statistics for each player (strategy) in the form of boxplots, and the history of moves related to the last repetition.

Available options:
```
--nplay NPLAY  number of players in the game (default: 8)
--niter NITER  number of repetitions for each match (default: 50)
--nrep NREP    number of repetitions of the game (default: 10)
--seed [SEED]  seed for the PRNG (not issued = clock-based; no number = 100 is used)
--fixed        fix Mainly bad/good probabilities to 0.75/0.25 (default: False)
--saveimg      save output images instead of showing them (default: False)
--latex        output dataframes as latex code (default: False)
```

In [ ]:
%matplotlib inline
%run code/ipd2p.py --seed

### IPD between multiple players (MPIPD)

The MPIPD is a simple generalization of the 2-player version seen above.

Given the number of players, we generate one strategy for each of them and setup a round-robin tournament: each player play against all the other participants.
Strategies are fixed throughout the whole tournament.

We iterate each match and the whole tournament multiple times to collect statistics, and show them using boxplots.

Available options:
```
--nplay NPLAY  number of players in the game (default: 50)
--niter NITER  number of repetitions for each match (default: 50)
--nrep NREP    number of repetitions of the game (default: 10)
--seed [SEED]  seed for the PRNG (not issued = clock-based; no number = 100 is used)
--fixed        fix Mainly bad/good probabilities to 0.75/0.25 (default: False)
--saveimg      save output images instead of showing them (default: False)
--latex        output dataframes as latex code (default: False)
```

In [ ]:
%matplotlib inline
%run code/ipdmp.py --seed

### Repeated Multiple Players IPD (rMPIPD)

The MPIPD shown above is repeated multiple times, but at each iteration the population changes.
We stop at convergence (more than 3/4 of the population has the same strategy) or if we reach a maximum number of iterations.

We implement two different ways:

1. The population remains constant: we add strategies that achieve good results and remove those that do not perform well. The percentage of players added and removed for each iteration is set before starting the game rounds.

2. The population increases: we start with one player per strategy and on each iteration we only add players with strategies that perform well in the previus iteration. We used three metrics to decide how to modify the population, all based on the sorted list of players' rank (score):
    - according to $i/num\_players$ where $i$ is the $i$-th player in the list
    - dividing the ordered population in three sets and using a different probability value for each of the three (add more from first set, less from the last)
    - according to each players' score, computed as $pl\_points/max(points\_list)$

Common available options:
```
--nplay NPLAY      number of players in the game (default: 50)
--niter NITER      number of repetitions for each match (default: 50)
--maxrep MAXREP    max number of allowed repetitions (default: 10)
--percent PERCENT  percentage of the population to be considered [if applicable] (default: 0.3)
--seed [SEED]      seed for the PRNG (not issued = clock-based; no number = 100 is used)
--fixed            fix Mainly bad/good probabilities to 0.75/0.25 (default: False)
--saveimg          save output images instead of showing them (default: False)
--latex            output dataframes as latex code (default: False)
```

#### 1. Constant Population

In [ ]:
%matplotlib inline
%run code/ripdmp_const_pop.py --seed

#### 2. Increasing population

Additional option to common ones:
```
--altern ALTERN  method to be used when changing population [details in the report] (default: 1)
```

In [ ]:
%matplotlib inline
%run code/ripdmp_incr_pop.py --seed

### rMPIPD with mutating strategies

Player's strategies are allowed to mutate according to a random parameter $c$ that encodes the attitude of cooperation of a player.

Again we propose two alternatives to update the gene:

1. the new $c_N$ is randomly generated

2. based on player's ranking: bad players obey to $c_N = (c+(i/num\_players)^2)/2$, (players high in the chart will go less cooperative and vice-versa). $1-i$ is instead used for good players, for the opposite behavior.

If $c_N$ differs significantly from the old $c$ value the strategy will change ($c=0.5$ as divider).
The new strategy is selected among a list of some default and some random $k$ values, bounded between $(1-c)\times 100$ and $\min(id,50)$ (more coop direction) or between $\max(id,50)$ and $(1-c)\times 100$ (less coop).


Available options:
```
--nplay NPLAY      number of players in the game (default: 50)
--niter NITER      number of repetitions for each match (default: 50)
--maxrep MAXREP    max number of allowed repetitions (default: 10)
--altern ALTERN    method to be used when changing population [details in the report] (default: 1)
--seed [SEED]      seed for the PRNG (not issued = clock-based; no number = 100 is used)
--fixed            fix Mainly bad/good probabilities to 0.75/0.25 (default: False)
--saveimg          save output images instead of showing them (default: False)
--latex            output dataframes as latex code (default: False)
```

In [ ]:
%matplotlib inline
%run code/cipdmp.py --seed